In [8]:
import sys
import os
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
from statistics import mean
import warnings
warnings.filterwarnings("ignore")
thisDir = os.path.expanduser('~/Desktop/Porteretal_taskprediction/')
dataDir = thisDir +  'data/corrmats/'

outDir = thisDir + 'output/results/Ridge/'
taskList=['pres1','pres2','pres3']

DS=pd.read_csv(outDir+'single_task/sep_mem_pres_accCG.csv',usecols=['train_task','test_task','diff_sub','same_sub'])
DS.rename(columns={'same_sub':'SS','diff_sub':'OS'},inplace=True)
single_task=pd.melt(DS,value_vars=['SS','OS'],var_name='Analysis',value_name='acc',id_vars=['train_task','test_task'])

def shuffle(df, n=1, axis=0):
    df['new']=np.random.permutation(df['acc'].values)
    null=df[df['Analysis']=='SS'].new.values-df[df['Analysis']=='OS'].new.values
    value=mean(null)
    return value
for task in taskList:
    for test_task in taskList:
        if task==test_task:
            continue
        idx = np.where((single_task['train_task']==task) & (single_task['test_task']==test_task) | (single_task['test_task']==task) & (single_task['train_task']==test_task))
        tmp=single_task.loc[idx]
        Diff=tmp[tmp['Analysis']=='SS'].acc.values-tmp[tmp['Analysis']=='OS'].acc.values
        TrueDiff=mean(Diff)
        distribution=[]
        for i in range(1000):
            null=shuffle(tmp)
            distribution.append(null)
        null=pd.DataFrame(distribution,columns=['distribution'])
        keep=null[null['distribution']>TrueDiff]
        count=keep['distribution'].count()
        p=(count+1)/1001
        print('p value for training '+task+ ' and ' +test_task+ ' is ' +str(p))

p value for training pres1 and pres2 is 0.000999000999000999
p value for training pres1 and pres3 is 0.000999000999000999
p value for training pres2 and pres1 is 0.000999000999000999
p value for training pres2 and pres3 is 0.000999000999000999
p value for training pres3 and pres1 is 0.000999000999000999
p value for training pres3 and pres2 is 0.000999000999000999


In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import numpy as np
import os
import sys
import pandas as pd
import itertools
from statistics import mean
import scipy.io
#import results
import warnings
warnings.filterwarnings("ignore")
# Initialization of directory information:
thisDir = os.path.expanduser('~/Desktop/Porteretal_taskprediction/')
dataDir = thisDir + 'data/corrmats/'
outDir = thisDir + 'output/results/'
figsDir=thisDir + 'output/figures/'
IndNetDir=thisDir+'data/IndNet/'
# Subjects and tasks
taskList=['pres1','pres2','pres3']
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
#all possible combinations of subs and tasks
subsComb=(list(itertools.permutations(subList, 2)))
tasksComb=(list(itertools.combinations(taskList, 2)))
DSvars=list(itertools.product(list(subsComb),list(taskList)))
splitDict=dict([('MSC01',10),('MSC02',10),('MSC03',8),('MSC04',10),('MSC05',10),('MSC06',9),('MSC07',9),('MSC10',10)])
clfDict=dict([('Ridge',RidgeClassifier()),('Log',LogisticRegression(solver='lbfgs')),('SVM',LinearSVC())])
taskDict=dict([('mem',0),('motor',1),('glass',2),('semantic',3)])
BSvars=list(itertools.product(list(subsComb),list(tasksComb))) #diff sub diff task

def matFiles(df='path'):
    """
    Convert matlab files into upper triangle np.arrays
    Parameters
    -----------
    df : str
        Path to file
    Returns
    -----------
    ds : 2D upper triangle FC measures in (roi, days) format

    """
    #Consistent parameters to use for editing datasets
    nrois=333
    #Load FC file
    fileFC=scipy.io.loadmat(df)

    #Convert to numpy array
    fileFC=np.array(fileFC['parcel_corrmat'])
    #Replace nans and infs with zero
    fileFC=np.nan_to_num(fileFC)
    nsess=fileFC.shape[2]
    #Index upper triangle of matrix
    mask=np.triu_indices(nrois,1)
    ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
    count=0
    #Loop through all 10 days to reshape correlations into linear form
    for sess in range(nsess):
        tmp=fileFC[:,:,sess]
        ds[count]=tmp[mask]
        count=count+1
    mask = (ds == 0).all(1)
    column_indices = np.where(mask)[0]
    df = ds[~mask,:]
    return df

def classifyPress():
    """
    Classifying different subjects (DS) along the same task

    Parameters
    -------------
    classifier : string
        Provide classifier type for analysis, options SVM=LinearSVC(), Log=LogisticRegression(solver='lbfgs'), Ridge=RidgeClassifier()

    Returns
    -------------
    dfDS : DataFrame
        Dataframe consisting of average accuracy across all subjects

    """
    clf=RidgeClassifier()
    same_sub_per_task=[] #SS same sub
    diff_sub_per_task=[] #OS other sub
    tmp_BS=pd.DataFrame(BSvars, columns=['sub','task'])
    dfDS=pd.DataFrame()
    dfDS[['train_task','test_task']]=pd.DataFrame(tmp_BS['task'].tolist())
    dfDS[['train_sub', 'test_sub']]=pd.DataFrame(tmp_BS['sub'].tolist())

    for index, row in dfDS.iterrows():
        taskFC=matFiles(dataDir+'mem/'+row['train_task']+'/'+row['train_sub']+'_parcel_corrmat.mat')
        restFC=matFiles(dataDir+'mem/'+row['test_task']+'/'+row['train_sub']+'_parcel_corrmat.mat')
        test_taskFC=matFiles(dataDir+'mem/'+row['train_task']+'/'+row['test_sub']+'_parcel_corrmat.mat')
        test_restFC=matFiles(dataDir+'mem/'+row['test_task']+'/'+row['test_sub']+'_parcel_corrmat.mat')
        SSacc, OSacc=folds(clf, taskFC, restFC, test_taskFC, test_restFC)
        same_sub_per_task.append(SSacc)
        diff_sub_per_task.append(OSacc)
    SS=mean(same_sub_per_task)
    OS=mean(diff_sub_per_task)
    return SS, OS

def folds(clf,taskFC, restFC, test_taskFC, test_restFC):
    """
    Cross validation to train and test using nested loops

    Parameters
    -----------
    clf : obj
        Machine learning algorithm
    analysis : str
        Analysis type
    taskFC, restFC, test_taskFC, test_restFC : array_like
        Input arrays, training and testing set of task and rest FC
    Returns
    -----------
    total_score : float
        Average accuracy across folds
    acc_score : list
        List of accuracy for each outer fold
    """

    loo = LeaveOneOut()
    taskSize=taskFC.shape[0]
    restSize=restFC.shape[0]
    t = np.ones(taskSize, dtype = int)
    r=np.zeros(restSize, dtype=int)
    y_test_task=np.ones(test_taskFC.shape[0])
    y_test_rest=np.zeros(test_restFC.shape[0])
    ytest=np.concatenate((y_test_task,y_test_rest))
    Xtest=np.concatenate((test_taskFC,test_restFC))
    #Test same sub
    SS_acc=[]
    OS_acc=[]
    #fold each training set
    for train_index, test_index in loo.split(taskFC):
        Xtrain_rest,Xval_rest=restFC[train_index],restFC[test_index]
        Xtrain_task,Xval_task=taskFC[train_index], taskFC[test_index]
        ytrain_rest,yval_rest=r[train_index], r[test_index]
        ytrain_task,yval_task=t[train_index], t[test_index]
        X_tr=np.concatenate((Xtrain_task, Xtrain_rest))
        y_tr = np.concatenate((ytrain_task,ytrain_rest))
        X_val=np.concatenate((Xval_task, Xval_rest))
        y_val = np.concatenate((yval_task,yval_rest))
        y_tr=np.random.permutation(y_tr)
        clf.fit(X_tr,y_tr)
        SSscores=clf.score(X_val,y_val)
        SS_acc.append(SSscores)
        OSscores=clf.score(Xtest,ytest)
        OS_acc.append(OSscores)
    OStotal_acc=mean(OS_acc)
    SStotal_acc=mean(SS_acc)
    return SStotal_acc, OStotal_acc
def permut_single_task():
    SSscore=[]
    OSscore=[]
    for i in range(1000):
        SS,OS=classifyPress()
        SSscore.append(SS)
        OSscore.append(OS)
        print(str(i))
    ALL_perms=pd.DataFrame()
    ALL_perms['SS']=SSscore
    ALL_perms['OS']=OSscore
    ALL_perms.to_csv(outDir+'Ridge/single_task/permutation_memPres.csv',index=False)

In [ ]:
permut_single_task()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27